In [ ]:
import cv2
import numpy as np
from pathlib import Path


def detect_and_crop_aruco(image_path, output_path=None, padding=5, expected_ids=None):
    """
    Detect ArUco markers and crop with custom boundaries.
    Uses error correction and ID filtering for damaged/worn markers.
    
    Crop boundaries:
    - Below the top two markers
    - Above the bottom two markers  
    - To the right of the left two markers (inner edge)
    - All the way to the right edge of the right two markers (outer edge)
    """
    if expected_ids is None:
        expected_ids = {0, 1, 2, 3}
    
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Setup ArUco detector with error correction for damaged markers
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    parameters = cv2.aruco.DetectorParameters()
    
    # Critical: allow markers at image edge
    parameters.minDistanceToBorder = 0
    
    # Error correction for damaged/worn markers
    parameters.errorCorrectionRate = 1.0
    
    # Adaptive threshold tuning - key for reliable detection
    parameters.adaptiveThreshWinSizeMin = 3
    parameters.adaptiveThreshWinSizeMax = 153
    parameters.adaptiveThreshWinSizeStep = 10
    
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, rejected = detector.detectMarkers(gray)
    
    # Filter to only expected IDs
    filtered_corners = []
    filtered_ids = []
    
    if ids is not None:
        for corner, marker_id in zip(corners, ids):
            if marker_id[0] in expected_ids:
                filtered_corners.append(corner)
                filtered_ids.append(marker_id)
    
    if len(filtered_ids) < 4:
        raise ValueError(f"Need 4 markers, found: {len(filtered_ids)} (IDs: {[id[0] for id in filtered_ids]})")
    
    corners = filtered_corners
    ids = np.array(filtered_ids)
    
    # Organize markers by position
    marker_data = []
    for corner, marker_id in zip(corners, ids):
        center = corner[0].mean(axis=0)
        marker_data.append({
            'id': marker_id[0],
            'corners': corner[0],
            'center': center
        })
    
    # Sort by Y to get top vs bottom, then by X to get left vs right
    marker_data.sort(key=lambda m: m['center'][1])
    top_markers = sorted(marker_data[:2], key=lambda m: m['center'][0])
    bottom_markers = sorted(marker_data[2:], key=lambda m: m['center'][0])
    
    top_left = top_markers[0]
    top_right = top_markers[1]
    bottom_left = bottom_markers[0]
    bottom_right = bottom_markers[1]
    
    def get_marker_bounds(marker):
        c = marker['corners']
        return {
            'left': c[:, 0].min(),
            'right': c[:, 0].max(),
            'top': c[:, 1].min(),
            'bottom': c[:, 1].max()
        }
    
    tl = get_marker_bounds(top_left)
    tr = get_marker_bounds(top_right)
    bl = get_marker_bounds(bottom_left)
    br = get_marker_bounds(bottom_right)
    
    # Calculate crop boundaries
    left_boundary = max(tl['right'], bl['right']) + padding
    right_boundary = max(tr['right'], br['right']) - padding
    top_boundary = max(tl['bottom'], tr['bottom']) + padding
    bottom_boundary = min(bl['top'], br['top']) - padding
    
    x1 = int(max(0, left_boundary))
    x2 = int(min(image.shape[1], right_boundary))
    y1 = int(max(0, top_boundary))
    y2 = int(min(image.shape[0], bottom_boundary))
    
    # Crop
    cropped = image[y1:y2, x1:x2]
    
    # Save
    if output_path:
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(output_path), cropped)
        
        # Save visualization
        viz = image.copy()
        cv2.aruco.drawDetectedMarkers(viz, corners, ids)
        cv2.rectangle(viz, (x1, y1), (x2, y2), (0, 255, 0), 3)
        viz_path = output_path.parent / f"{output_path.stem}_viz{output_path.suffix}"
        cv2.imwrite(str(viz_path), viz)
    
    return cropped


def process_batch(input_dir, output_dir, padding=5):
    """Process all images in a directory."""
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
    images = [f for f in input_dir.iterdir() if f.suffix in extensions]
    
    print(f"Found {len(images)} images to process")
    
    success = 0
    failed = 0
    
    for img_path in images:
        print(f"\nProcessing: {img_path.name}")
        try:
            output_path = output_dir / f"{img_path.stem}_cropped{img_path.suffix}"
            detect_and_crop_aruco(img_path, output_path, padding=padding)
            print(f"  ✓ Success")
            success += 1
        except Exception as e:
            print(f"  ✗ Error: {e}")
            failed += 1
    
    print(f"\n{'='*60}")
    print(f"Successful: {success}/{len(images)}")
    print(f"Failed: {failed}/{len(images)}")

In [5]:
if __name__ == "__main__":
    from pathlib import Path
    
    data_dir = Path("data")
    output_dir = Path("out")
    output_dir.mkdir(exist_ok=True)
    
    # Get all image files
    image_extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
    image_files = [f for f in data_dir.iterdir() if f.suffix in image_extensions]
    
    print(f"Found {len(image_files)} images to process\n")
    
    successful = 0
    failed = 0
    
    for image_file in sorted(image_files):
        print(f"\n{'='*60}")
        print(f"Processing: {image_file.name}")
        print('='*60)
        
        output_path = output_dir / f"cropped_{image_file.stem}.png"
        
        try:
            cropped = detect_and_crop_aruco(
                str(image_file),
                str(output_path),
                padding=5
            )
            print(f"✓ Success! Cropped shape: {cropped.shape}")
            successful += 1
        except Exception as e:
            print(f"✗ Error processing {image_file.name}: {e}")
            failed += 1
    
    print(f"\n{'='*60}")
    print(f"Processing complete!")
    print(f"Successful: {successful}/{len(image_files)}")
    print(f"Failed: {failed}/{len(image_files)}")
    print(f"Output directory: {output_dir.absolute()}")
    print('='*60)

Found 12 images to process


Processing: UCSB-IZC00055302_L.JPG
✓ Success! Cropped shape: (620, 1739, 3)

Processing: UCSB-IZC00055302_R.JPG
✓ Success! Cropped shape: (612, 1734, 3)

Processing: UCSB-IZC00055338_L.JPG
✓ Success! Cropped shape: (992, 1995, 3)

Processing: UCSB-IZC00055338_R.JPG
✗ Error processing UCSB-IZC00055338_R.JPG: Need 4 markers, found: 3 (IDs: [np.int32(0), np.int32(1), np.int32(2)])

Processing: UCSB-IZC00065365_L.JPG
✓ Success! Cropped shape: (1416, 2708, 3)

Processing: UCSB-IZC00065365_R.JPG
✗ Error processing UCSB-IZC00065365_R.JPG: Need 4 markers, found: 3 (IDs: [np.int32(2), np.int32(3), np.int32(0)])

Processing: UCSB-IZC00065388_L.JPG
✓ Success! Cropped shape: (1420, 2721, 3)

Processing: UCSB-IZC00065388_R.JPG
✗ Error processing UCSB-IZC00065388_R.JPG: Need 4 markers, found: 3 (IDs: [np.int32(0), np.int32(3), np.int32(1)])

Processing: UCSB-IZC00065413_L.JPG
✓ Success! Cropped shape: (613, 1738, 3)

Processing: UCSB-IZC00065413_R.JPG
✗ Error processing 